In [ ]:
from bs4 import BeautifulSoup
from requests import get
import re
import csv
import requests
import urllib
import pandas as pd
import re
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
print("Current Timestamp...",timestr)

def set_var(soup):
    
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    viewstategen = soup.select("#__VIEWSTATEGENERATOR")[0]['value']
    eventvalidation = soup.select("#__EVENTVALIDATION")[0]['value']
    
    return viewstate,viewstategen, eventvalidation

fal_list = []

def open_url(formData,url,headers):
    encodedFields = urllib.parse.urlencode(formData)
    # second HTTP request with form data
    encodedFields = encodedFields.encode('ascii')
    new_url = url.replace('http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=', 'http://icds-wcd.nic.in/icdsmis_ViewCurMonMPR.aspx?awcid=')
    f = urllib.request.urlopen(urllib.request.Request(new_url,data = encodedFields,headers = headers))
    soup = BeautifulSoup(f,"html.parser")
    return soup, new_url


def gen_traversal(var, soup, url, locid = ''):
    headers = {
        'Referer': url
    , 'Content-Type': 'application/x-www-form-urlencoded'
    , 'Cookie': 'ASP.NET_SessionId=iolwllq1ajmvwp40dziwswt1'
    , 'Host': 'icds-wcd.nic.in'
    , 'Connection': 'keep-alive'
    , 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
    , 'Accept-Encoding': 'gzip, deflate, br'
    , 'Accept-Language': 'en-US,en;q=0.5'
    , 'Origin': 'https://icds-wcd.nic.in'
    , 'Upgrade-Insecure-Requests': '1'
    , 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
    }
    viewstate,viewstategen, eventvalidation = set_var(soup)
    
    formData = (
        ('__VIEWSTATE', viewstate),
        ('__VIEWSTATEGENERATOR', viewstategen),
        ('__EVENTTARGET', 'View AW-MPR (Current)'),
#        ('__EVENTTARGET', LinkButton2),
#        ('__EVENTTARGET', var),
#        (var, locid),
        ('__EVENTVALIDATION', eventvalidation)
    )
    
    new_soup, new_url = open_url(formData,url,headers)
    return formData, new_soup,new_url

def extract_form_fields(soup):
    "Turn a BeautifulSoup form in to a dict of fields and default values"
    fields = {}
    for p in soup.findAll('p'):
        value = ''
        value = p.text
        fields = [value]
        continue    
            
    for span in soup.findAll('span'):
        if not span.has_attr('id'):
            continue
        else:
            value = ''
            value = span.text
            fields[span['id']] = value
            continue    
    
    for input in soup.findAll('input'):
        # ignore submit/image with no name attribute
        if input['type'] in ('submit', 'image') and not 'name' in input:
            continue

        # ignore type hidden
        if input['type'] in ('hidden'):
            continue

        # single element nome/value fields
        if input['type'] in ('text', 'hidden', 'password', 'submit', 'image'):
            value = ''
            if input.has_attr('value'):
                value = input['value']
            fields[input['name']] = value
            continue

        # checkboxes and radios
        if input['type'] in ('checkbox', 'radio'):
            value = ''
            if input.has_attr('checked'):
                if input.has_attr('value'):
                    value = input['value']
                fields[input['name']] = value
                continue

    # textareas
    for textarea in soup.findAll('textarea'):
        fields[textarea['name']] = textarea.string or ''

    # select fields
    for select in soup.findAll('select'):
        value = ''
        options = select.findAll('option')
        is_multiple = select.has_key('multiple')
        selected_options = [
            option for option in options
            if option.has_key('selected')
        ]

        if not selected_options and options:
            selected_options = [options[0]]

        if not is_multiple:
            assert(len(selected_options) < 2)
            if len(selected_options) == 1:
                value = selected_options[0]['value']
        else:
            value = [option['value'] for option in selected_options]

        fields[select['name']] = value

    return fields

def extract_awc_info(infile,times_try):

    # read awc file and extract data for each awc
    # fetch url1, if attempt fails, pause and try again, if second attempt fails then write to error file
    # build url2 with form data
    # fetch url2, if attempt fails, pause and try again, if second attempt fails then write to error file
    
    #file_to_open = "awc_" + str(dist_process_code) + ".csv" 

    awc_data = []
    url_fail = []
        
    with open(infile, 'r', encoding="utf-8") as csvfile:
        next(csvfile) # skip header
        f_awc = csv.reader(csvfile, delimiter=',')
        count = 0
        count_url1_fail = 0
        count_url2_fail = 0
        
        for row in f_awc:
            count = count + 1        
            
            # test for only limited records
            #if count > 50:
            #    print("test condition reached...break")
            #    break
            
            awc_code = row[0]
            awc_code = re.sub('[^0-9]','', awc_code)

        # get url1
            url1 = "http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=" +str(awc_code).strip() + "&State_Code=" +str(awc_code.strip()[0:2]).strip() + "&District_Code=" +str(awc_code.strip()[2:5]).strip() + "&Project_Code=" +str(awc_code.strip()[0:7])
            print(url1)       
        
            try:
                content = get(url1).content
            except:
                print("Connection refused by the server for url1....", awc_code)
                time.sleep(5)
                try:
                    content = get(url1).content
                    count_url1_fail = count_url1_fail + 1
                    url_fail.append([awc_code])
                except:
                    print("Second try failed for url1....", awc_code)                
                    continue

            
            soup1 = BeautifulSoup(content, "lxml")
        
            # get url2
            url = url1
            soup = soup1
            var = ''
            
            try:
                f, soup2,url2 = gen_traversal(var,soup,url)
            except:
                print("Connection refused by the server for url2....", awc_code)
                time.sleep(10)

                try:
                    f, soup2,url2 = gen_traversal(var,soup,url)
                except:
                    print("Second try failed for url2....", awc_code)
                    count_url2_fail = count_url2_fail + 1
                    url_fail.append([awc_code])            
                    continue
                                        
            #print(url2)
        
            # get form data from url1
            form_data = {'awc_code': awc_code.strip()}
            form_data1 = {}
            form_data1 = extract_form_fields(soup1)
            form_data.update(form_data1)
        
            # get form data from url2
            form_data2 = {}
            form_data2 = extract_form_fields(soup2)
            form_data.update(form_data2)
            awc_data.append(form_data)
            
            # print record count at every 50 records processed
            if (count%50 == 0): 
                print("==== Intermediate Count ====", count)
                print("==== URL1 failed        ====", count_url1_fail)
                print("==== URL2 failed        ====", count_url2_fail)
            

    print("==== Total Count        ====", count)
    print("==== URL1 failed        ====", count_url1_fail)
    print("==== URL2 failed        ====", count_url2_fail)

    # write to csv file
    df = pd.DataFrame(awc_data)
    df = df.rename(columns = {'lblState':'State_1', 'lblDistrict':'District_1', 'lblProject':'Project_1'})
    df.columns = df.columns.str.replace('lbl', '')
    df.columns = df.columns.str.replace('txt', '')
    df.columns = df.columns.str.replace('dd', '')
    df.columns = df.columns.str.replace('opt', '')
    outfile = 'awc_monitoring_details_' + str(times_try) + '_' +str(awc_code.strip()[2:5]).strip() + '.csv'
    df.to_csv(outfile, index=False)

    # write failed awc_code to .csv file
    process_failed_recs = 'n'
    if url_fail != []:
        process_failed_recs = 'y'
        df_fail = pd.DataFrame(url_fail)
        df_fail.columns = ['awc_code']
        df_fail = df_fail.drop_duplicates()
        failed_file = 'awc_failed_' + str(times_try) + '_' +str(awc_code.strip()[2:5]).strip() + '.csv'
        df_fail.to_csv(failed_file, index=False)
        print("==== Unique failed      ====", df_fail.shape[0])
        
    return process_failed_recs, df

# The code below fetches the AWC co-ordinates for all districts of Karnataka
print("Fetching AWC co-ordinates for all districts")
for dist_code in range(555,585):
    dist_data = []
    url_fail = []

    url1 = "http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=" +str(dist_code).strip() 
    print(url1)

    try:
        content = get(url1).content
    except:
        print("Connection refused by the server for url1....", dist_code)

            
    soup1 = BeautifulSoup(content, "lxml")

    for p in soup1.findAll('p'):
        filedata = ''
        filedata = p.text
           
    newdata = filedata.replace('[[', '')
    newdata = newdata.replace(']]', '')
    newdata = newdata.replace('],[', '\n')
    newdata = newdata.replace("'", '"')  

    outfile = 'awc_'+str(dist_code).strip() + '.txt'
    f = open(outfile,'w')
    f.write(newdata)
    f.close()

    outcsv = 'awc_'+str(dist_code).strip() + '.csv'
    df_awc = pd.read_csv(outfile, header=None)
    df_awc.columns=['awc_name', 'lat', 'lon', 'awc_code', 'awc_alpha_code']
    df_awc['dist_code'] = '"' + str(dist_code) + '"'
    first_column = df_awc.pop('awc_code')
    df_awc.insert(0, 'awc_code', first_column) # makde awc_code the first column
    df_awc.to_csv(outcsv, index=False)

# Combine all district files and save as state file with current timestamp
df_concat = pd.DataFrame()

for i in range(555,585):
    infile = 'awc_' + str(i) + '.csv'
    df_infile = pd.read_csv(infile)
    df_concat = pd.concat([df_concat, df_infile])

# strip quotation marks from all columns
df_concat[df_concat.columns] = df_concat.apply(lambda x: x.str.strip())
df_concat[df_concat.columns] = df_concat.apply(lambda x: x.str.strip('\"'))

print('Total AWC in the state....', df_concat.shape[0])
outfile = 'awc_karnataka_districts_' + str(timestr) + '.csv'

df_concat.to_csv(outfile, index=False)

# Read input file awc_dist_code_to_process.csv to find out which districts to process
# Input file has 2 fields: 1) input_dist_code 2)input_process_y_n; edit this file to control which districts to process
# Each AWC has two urls from which we extract data. For example for AWC = 29572011016, the urls are:
# 1) http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29572011016&State_Code=29&District_Code=572&Project_Code=2957201
# 2) http://icds-wcd.nic.in/icdsmis_ViewCurMonMPR.aspx?awcid=29572011016&State_Code=29&District_Code=572&Project_Code=2957201
# The extraction for each AWC will be attempted for a max of three times in the below code
# Records not extracted after three attempts if any will be written to awc_failed_third_try_ + district code + timestamp

file_to_open1 = "awc_dist_code_to_process.csv" 
        
with open(file_to_open1, 'r', encoding="utf-8") as csvfile1:
    next(csvfile1) # skip header
    dist_process = csv.reader(csvfile1, delimiter=',')
    
    for dist_process_row in dist_process:
        
        if (dist_process_row[1] == 'y' or dist_process_row[1] == 'Y'):
            dist_process_code = dist_process_row[0]
            print("Now processing district...", dist_process_row[0])
            infile = "awc_" + str(dist_process_code) + ".csv" 
            times_try='first_try'
            print("Attempting first try")
            process_failed_recs, df = extract_awc_info(infile,times_try)
            df_first = df.copy()
            
            if process_failed_recs == 'y':
                infile = 'awc_failed_' + str(times_try) + '_' + str(dist_process_code) + '.csv'
                times_try='second_try'
                print("Attempting second try")
                process_failed_recs, df = extract_awc_info(infile,times_try)
                df_second = df.copy()
                
                if process_failed_recs == 'y':
                    infile = 'awc_failed_' + str(times_try) + '_' + str(dist_process_code) + '.csv'
                    times_try='third_try'
                    print("Attempting third try")
                    process_failed_recs, df = extract_awc_info(infile,times_try)
                    df_third = df.copy()
                    df_final = pd.concat([df_first, df_second, df_third], ignore_index=True, sort=False)
                    
                    if process_failed_recs == 'y':
                        infile = 'awc_failed_' + str(times_try) + '_' + str(dist_process_code) + '.csv'
                        print("!!!Records not processed after three attempts. Unprocessed records are in file...",infile)
                else:
                    print("All records processed after the second try")
                    df_final = pd.concat([df_first, df_second], ignore_index=True, sort=False)
                    
            else:
                print("All records processed in the first try")
                df_final = df_first.copy()
                    
            file_final = 'awc_monitoring_details_final_' + str(dist_process_code) + '_' + str(timestr) + '.csv'
            df_final.to_csv(file_final, index=False)  
            print('Final file has....', df_final.shape[0])
            
#====== code ends here ======

D:\2020\Anaconda\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Current Timestamp... 20220428-131432
Fetching AWC co-ordinates for all districts
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=555
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=556
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=557
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=558
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=559
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=560
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=561
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=562
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=563
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId=564
http://awcka.indiagis.org/icdska/gmap/lyr.getGData?mapRltd=DT&objRltd=PAW&mapId

D:\2020\Anaconda\lib\site-packages\bs4\element.py:1890: UserWarning: has_key is deprecated. Use has_attr("multiple") instead.
  warnings.warn('has_key is deprecated. Use has_attr("%s") instead.' % (
D:\2020\Anaconda\lib\site-packages\bs4\element.py:1890: UserWarning: has_key is deprecated. Use has_attr("selected") instead.
  warnings.warn('has_key is deprecated. Use has_attr("%s") instead.' % (


http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010102&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010103&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010104&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010105&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010106&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010107&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010108&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010109&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010310&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010311&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010312&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010313&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010314&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010315&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010316&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555010317&State_Code=29&District_Code=555&Project_Code=2955501
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

...

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060117&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060201&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060202&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060203&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060204&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060205&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060206&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060207&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060430&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060431&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060501&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060502&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060503&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060504&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060505&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060506&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060706&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060707&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060708&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060709&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060710&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060711&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060712&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555060713&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061008&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061009&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061010&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061011&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061012&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061013&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061014&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555061015&State_Code=29&District_Code=555&Project_Code=2955506
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295

http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070227&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070228&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070301&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070302&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070303&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070304&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070305&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=29555070306&State_Code=29&District_Code=555&Project_Code=2955507
http://icds-wcd.nic.in/icdsknowawcformat.aspx?awcode=295